## 🖼️🧩 Step 1: Gallery Data Modeling

### Import Libraries

In [57]:
# Import Libraries
import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

### Create the tag to pull galleries from tourism section

In [2]:
tags = {
        "tourism": "gallery"
       }

### Fetch Berlin Geometries

In [3]:
gallery_raw = ox.features_from_place("Berlin, Germany", tags)


### Display basic info

In [4]:
print(f"Number of gallery entries fetched: {len(gallery_raw)}")
gallery_raw.head()

Number of gallery entries fetched: 322


geometry addr:city addr:country  \
element id                                                            
node    301107444  POINT (13.57134 52.40975)    Berlin           DE   
        370766098  POINT (13.31363 52.50313)    Berlin           DE   
        410135935  POINT (13.42749 52.49333)    Berlin           DE   
        410692505  POINT (13.43706 52.49291)       NaN          NaN   
        410745800  POINT (13.37533 52.55509)    Berlin           DE   

                  addr:housenumber addr:postcode     addr:street  \
element id                                                         
node    301107444                6         12524   Richterstraße   
        370766098               57         10629   Mommsenstraße   
        410135935               31         12047   Bürknerstraße   
        410692505              NaN           NaN             NaN   
        410745800               16         13357  Schwedenstraße   

                      addr:suburb   contact:phone  \
element id                                          
node    301107444       Bohnsdorf  +49 30 6764261   
        370766098  Charlottenburg             NaN   
        410135935        Neukölln             NaN   
        410692505             NaN             NaN   
        410745800   Gesundbrunnen   +491708089110   

                                         contact:website     craft  ...  \
element id                                                          ...   
node    301107444                  http://achimkuehn.de/  sculptor  ...   
        370766098           http://www.galeriezandi.com/       NaN  ...   
        410135935                                    NaN       NaN  ...   
        410692505                                    NaN       NaN  ...   
        410745800  http://www.kronenboden.de/?page_id=10       NaN  ...   

                  heritage:operator lda:criteria ref:lda roof:levels  \
element id                                                             
node    301107444               NaN          NaN     NaN         NaN   
        370766098               NaN          NaN     NaN         NaN   
        410135935               NaN          NaN     NaN         NaN   
        410692505               NaN          NaN     NaN         NaN   
        410745800               NaN          NaN     NaN         NaN   

                  roof:shape building:use colour indoor type name:ko  
element id                                                            
node    301107444        NaN          NaN    NaN    NaN  NaN     NaN  
        370766098        NaN          NaN    NaN    NaN  NaN     NaN  
        410135935        NaN          NaN    NaN    NaN  NaN     NaN  
        410692505        NaN          NaN    NaN    NaN  NaN     NaN  
        410745800        NaN          NaN    NaN    NaN  NaN     NaN  

[5 rows x 106 columns]

### Save the raw data to a csv file

- Define file paths

In [5]:
raw_csv_path = "../sources/csv_files/gallery_raw.csv"
raw_geojson_path = "../sources/geojson_files/gallery_raw.geojson"

- Save csv & geojson to correct folders in sources

In [6]:
gallery_raw.to_csv(raw_csv_path, index=False )

In [7]:
gallery_raw.to_file(raw_geojson_path, driver="GeoJSON")

### Remove the columns with 75% or more missing data

In [8]:
# Step 1: Calculate percentage of missing values per column
missing_percent = gallery_raw.isnull().mean() * 100

# Step 2: Identify columns with less than 75% missing
columns_to_keep = missing_percent[missing_percent < 75].index

# Step 3: Create a new filtered DataFrame
gallery_filtered = gallery_raw[columns_to_keep]

# Check shape and preview
print(gallery_filtered.shape)
gallery_filtered.head()

(322, 13)


geometry addr:city addr:country  \
element id                                                            
node    301107444  POINT (13.57134 52.40975)    Berlin           DE   
        370766098  POINT (13.31363 52.50313)    Berlin           DE   
        410135935  POINT (13.42749 52.49333)    Berlin           DE   
        410692505  POINT (13.43706 52.49291)       NaN          NaN   
        410745800  POINT (13.37533 52.55509)    Berlin           DE   

                  addr:housenumber addr:postcode     addr:street  \
element id                                                         
node    301107444                6         12524   Richterstraße   
        370766098               57         10629   Mommsenstraße   
        410135935               31         12047   Bürknerstraße   
        410692505              NaN           NaN             NaN   
        410745800               16         13357  Schwedenstraße   

                      addr:suburb                   name  tourism  fee  \
element id                                                               
node    301107444       Bohnsdorf     Atelier Achim Kühn  gallery  NaN   
        370766098  Charlottenburg          Galerie Zandi  gallery   no   
        410135935        Neukölln  Studio für Gestaltung  gallery   no   
        410692505             NaN              La Girafe  gallery   no   
        410745800   Gesundbrunnen            kronenboden  gallery  NaN   

                                       opening_hours wheelchair website  
element id                                                               
node    301107444                                NaN        NaN     NaN  
        370766098  Mo-Fr 11:00-18:00; Sa 11:00-16:00         no     NaN  
        410135935  Th,Fr 13:00-18:00; Sa 12:00-16:00        yes     NaN  
        410692505                                NaN         no     NaN  
        410745800                                NaN        yes     NaN

### Check no Geometries missing

In [9]:
print("Missing geometries:", gallery_filtered.geometry.isna().sum())

Missing geometries: 0


### Add Latitude & Longitude columns to filtered geodataframe

In [10]:
# Reproject and extract lat/lon
gallery_filtered = gallery_filtered.to_crs(epsg=4326)
gallery_filtered['geometry'] = gallery_filtered['geometry'].apply(
    lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point()
)
gallery_filtered['latitude'] = gallery_filtered.geometry.y
gallery_filtered['longitude'] = gallery_filtered.geometry.x

### Verify the lat/lon amounts are correct

In [11]:
print("Latitude range:", gallery_filtered["latitude"].min(), "to", gallery_filtered["latitude"].max())

print("Longitude range:", gallery_filtered["longitude"].min(), "to", gallery_filtered["longitude"].max())

Latitude range: 52.4097499 to 52.6339917
Longitude range: 13.1459292 to 13.615677950110676


### Reset index and change column type 'id' to string

In [12]:
gallery_filtered = gallery_filtered.reset_index()

# set the id to string
gallery_filtered["id"] = gallery_filtered["id"].astype(str)
#  
# Drop the redundant column "element"
gallery_filtered= gallery_filtered.drop(columns=["element"],errors='ignore')
# Show final list of columns
print(gallery_filtered.columns.tolist())

['id', 'geometry', 'addr:city', 'addr:country', 'addr:housenumber', 'addr:postcode', 'addr:street', 'addr:suburb', 'name', 'tourism', 'fee', 'opening_hours', 'wheelchair', 'website', 'latitude', 'longitude']


In [13]:
gallery_filtered.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id                322 non-null    object  
 1   geometry          322 non-null    geometry
 2   addr:city         189 non-null    object  
 3   addr:country      125 non-null    object  
 4   addr:housenumber  220 non-null    object  
 5   addr:postcode     203 non-null    object  
 6   addr:street       220 non-null    object  
 7   addr:suburb       128 non-null    object  
 8   name              311 non-null    object  
 9   tourism           322 non-null    object  
 10  fee               140 non-null    object  
 11  opening_hours     143 non-null    object  
 12  wheelchair        118 non-null    object  
 13  website           143 non-null    object  
 14  latitude          322 non-null    float64 
 15  longitude         322 non-null    float64 
dtypes: float64(2), geo

---

## 🖼️🔄 Step 2: Gallery Data Transformation

### Drop unnecessary columns
    - Dont need Berlin and DE as user knows they are looking for data in Berlin, Germany
    - Drop tourism column as all Gallery
    - Drop suburb as it is the same as neighborhood which will be added later to make sure nothing is missing

In [14]:
gallery_filtered.drop(columns=['addr:city', 'addr:country', 'tourism', 'addr:suburb'], errors='ignore', inplace=True)


### Copy the filtered database for record purposes and then standardise column names

In [15]:
gallery_cleaned = gallery_filtered.copy()

def clean_column(col):
    col = col.strip().lower().replace('addr:', '')
    col = col.replace(' ', '_')
    col = re.sub(r'[^a-z0-9_]', '', col)
    return col

gallery_cleaned.columns = [clean_column(col) for col in gallery_cleaned.columns]

### Rename columns for clarity

In [16]:
gallery_cleaned.rename(columns={
    'postcode': 'postal_code',
    'housenumber': 'house_number'
}, inplace=True)

In [17]:
gallery_cleaned.columns.tolist()

['id',
 'geometry',
 'house_number',
 'postal_code',
 'street',
 'name',
 'fee',
 'opening_hours',
 'wheelchair',
 'website',
 'latitude',
 'longitude']

### Check the unique values in yes/no columns to see if possible to change to True/False

In [18]:
print("Unique fee:"  + str(gallery_cleaned.fee.unique()))                   # Can change as only yes/no
print("Unique wheelchair:"  + str(gallery_cleaned.wheelchair.unique()))     # Leave as is because of limited

Unique fee:[nan 'no' 'yes']
Unique wheelchair:[nan 'no' 'yes' 'limited']


In [19]:
gallery_cleaned["fee"] = gallery_cleaned["fee"].map({"yes": True, "no": False})

### Fetch Districts & Neighbourhoods

- Load official Berlin districts GeoDataFrame

In [20]:
districts_gdf = gpd.read_file("../sources/geojson_files/lor_ortsteile.geojson")

In [21]:
districts_gdf.head(2)

,gml_id,spatial_name,spatial_alias,spatial_type,OTEIL,BEZIRK,FLAECHE_HA,geometry
0,re_ortsteil.0101,0101,Mitte,Polygon,Mitte,Mitte,1063.8748,"POLYGON ((13.41649 52.52696, 13.41635 52.52702..."
1,re_ortsteil.0102,0102,Moabit,Polygon,Moabit,Mitte,768.7909,"POLYGON ((13.33884 52.51974, 13.33884 52.51974..."


- Reproject GeoDataFrames to EPSG:4326 

In [22]:
gallery_cleaned = gallery_cleaned.to_crs(epsg=4326)
districts_gdf = districts_gdf.to_crs(epsg=4326)

- Spatial join with district(Bezirk) and Neighbourhood_id(spatial_name)

In [23]:
gallery_df_district = gpd.sjoin(
    gallery_cleaned,
    districts_gdf[["BEZIRK", "spatial_name","geometry"]],
    how="left",
    predicate="within"
)

- Rename columns for clarity

In [24]:

gallery_df_district = gallery_df_district.rename(columns={
    "BEZIRK": "district",
    "spatial_name": "neighborhood_id"
}).drop(columns=["index_right"])  # drop district_number if not needed

- District mapping (official codes as strings)

In [25]:
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column (string)
gallery_df_district['district_id'] = gallery_df_district['district'].map(district_mapping).astype(str)

In [26]:
gallery_df_district.head()

,id,geometry,house_number,postal_code,street,name,fee,opening_hours,wheelchair,website,latitude,longitude,district,neighborhood_id,district_id
0,301107444,POINT (13.57134 52.40975),6,12524,Richterstraße,Atelier Achim Kühn,NaN,NaN,NaN,NaN,52.409750,13.571342,Treptow-Köpenick,0908,11009009
1,370766098,POINT (13.31363 52.50313),57,10629,Mommsenstraße,Galerie Zandi,False,Mo-Fr 11:00-18:00; Sa 11:00-16:00,no,NaN,52.503135,13.313631,Charlottenburg-Wilmersdorf,0401,11004004
2,410135935,POINT (13.42749 52.49333),31,12047,Bürknerstraße,Studio für Gestaltung,False,"Th,Fr 13:00-18:00; Sa 12:00-16:00",yes,NaN,52.493327,13.427490,Neukölln,0801,11008008
3,410692505,POINT (13.43706 52.49291),NaN,NaN,NaN,La Girafe,False,NaN,no,NaN,52.492914,13.437057,Friedrichshain-Kreuzberg,0202,11002002
4,410745800,POINT (13.37533 52.55509),16,13357,Schwedenstraße,kronenboden,NaN,NaN,yes,NaN,52.555091,13.375326,Mitte,0106,11001001


### Used reverse Geocoding and Nominatim to get Postal code & Street info
    - Created new columns at end to check against nulls in original columns

In [27]:
tqdm.pandas()

# Initialize geocoder
geolocator = Nominatim(user_agent="gallery_locator")

# Define function to extract postal code and street
def get_postcode_and_street(row):
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
        address = location.raw.get('address', {})
        postcode = address.get('postcode')
        street = address.get('road') or address.get('pedestrian') or address.get('footway') or address.get('street')
        return pd.Series([postcode, street])
    except:
        return pd.Series([None, None])

# Apply to your DataFrame
gallery_df_district[['postal_code_from_geo', 'street_from_geo']] = gallery_df_district.progress_apply(get_postcode_and_street, axis=1)


100%|██████████| 322/322 [11:25<00:00,  2.13s/it]


In [28]:
gallery_full = gallery_df_district.copy()

### Check null count before replacing Nulls in postal code

In [29]:
print(gallery_full[['postal_code']].isnull().sum())
print(gallery_full[['street']].isnull().sum())

postal_code    119
dtype: int64
street    102
dtype: int64


### Replace the Nulls in the original columns

In [30]:

gallery_full['postal_code'] = gallery_full['postal_code'].fillna(gallery_full['postal_code_from_geo'])
gallery_full['street'] = gallery_full['street'].fillna(gallery_full['street_from_geo'])

### Check null values now to see if a difference

In [31]:
print(gallery_full[['postal_code']].isnull().sum())
print(gallery_full[['street']].isnull().sum())

postal_code    26
dtype: int64
street    25
dtype: int64


### Data Cleaning & Normalization
    - Replace all missing values with Nan

In [32]:
# Replace empty strings with NaN
gallery_full.replace('', np.nan, inplace=True)

gallery_full.head(2)

,id,geometry,house_number,postal_code,street,name,fee,opening_hours,wheelchair,website,latitude,longitude,district,neighborhood_id,district_id,postal_code_from_geo,street_from_geo
0,301107444,POINT (13.57134 52.40975),6,12524,Richterstraße,Atelier Achim Kühn,NaN,NaN,NaN,NaN,52.409750,13.571342,Treptow-Köpenick,0908,11009009,12524,Richterstraße
1,370766098,POINT (13.31363 52.50313),57,10629,Mommsenstraße,Galerie Zandi,False,Mo-Fr 11:00-18:00; Sa 11:00-16:00,no,NaN,52.503135,13.313631,Charlottenburg-Wilmersdorf,0401,11004004,10629,Mommsenstraße


### Normalize the street name column

In [33]:
def normalize_street_name(name):
    if pd.isna(name):
        return np.nan
    # Replace underscores with spaces
    name = name.replace('_', ' ').replace('-', ' ')
    # Replace 'str.' or 'str' at end with ' Straße'
    name = re.sub(r'\bstr\.?\s*$', ' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'straße' if missing
    name = re.sub(r'(?<!\s)(straße)$', r' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'allee' if missing
    name = re.sub(r'(?<!\s)(allee)$', r' Allee', name, flags=re.IGNORECASE)
    # Ensure space before 'damm' if missing
    name = re.sub(r'(?<!\s)(damm)$', r' Damm', name, flags=re.IGNORECASE)
    # Ensure space before 'weg' if missing
    name = re.sub(r'(?<!\s)(weg)$', r' Weg', name, flags=re.IGNORECASE)
    # Ensure space before 'graben' if missing
    name = re.sub(r'(?<!\s)(graben)$', r' Graben', name, flags=re.IGNORECASE)
    # Ensure space before 'ufer' if missing
    name = re.sub(r'(?<!\s)(ufer)$', r' Ufer', name, flags=re.IGNORECASE)
    # Ensure space before 'korso' if missing
    name = re.sub(r'(?<!\s)(korso)$', r' Korso', name, flags=re.IGNORECASE)
    # Ensure space before 'zeile' if missing
    name = re.sub(r'(?<!\s)(zeile)$', r' Ziele', name, flags=re.IGNORECASE)
    # Ensure space before 'promenade' if missing
    name = re.sub(r'(?<!\s)(promenade)$', r' Promenade', name, flags=re.IGNORECASE)
    # Ensure space before 'kiez' if missing
    name = re.sub(r'(?<!\s)(kiez)$', r' Kiez', name, flags=re.IGNORECASE)
    # Ensure space before 'platz' if missing
    name = re.sub(r'(?<!\s)(platz)$', r' Platz', name, flags=re.IGNORECASE)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [34]:
gallery_full['street'] = gallery_full['street'].apply(normalize_street_name)

### Drop unnecessary columns

In [35]:
gallery_full.drop(columns=['geometry', 'district', 'postal_code_from_geo', 'street_from_geo'], errors='ignore', inplace=True)

### Convert all text in columns to lowercase to avoid any duplications

In [36]:
text_cols = ["name", "street", "website", "opening_hours", "wheelchair"]

for col in text_cols:
    if col in gallery_full.columns:
        gallery_full[col] = gallery_full[col].apply(
            lambda x: x.strip().lower() if isinstance(x, str) else x
        )

### Check column data types
    - All have correct type allocated so no need to change anything

In [37]:
gallery_full.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 322 entries, 0 to 321
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               322 non-null    object 
 1   house_number     220 non-null    object 
 2   postal_code      296 non-null    object 
 3   street           297 non-null    object 
 4   name             311 non-null    object 
 5   fee              140 non-null    object 
 6   opening_hours    143 non-null    object 
 7   wheelchair       118 non-null    object 
 8   website          143 non-null    object 
 9   latitude         322 non-null    float64
 10  longitude        322 non-null    float64
 11  neighborhood_id  322 non-null    object 
 12  district_id      322 non-null    object 
dtypes: float64(2), object(11)
memory usage: 35.2+ KB


### Remove duplicates
    - In this case we have no duplicates but will keep this check for future runs incase

In [38]:
# See how many duplicate rows exist
gallery_full.duplicated().sum()

np.int64(0)

In [39]:
# Display the actual duplicate rows
gallery_full[gallery_full.duplicated()]

,id,house_number,postal_code,street,name,fee,opening_hours,wheelchair,website,latitude,longitude,neighborhood_id,district_id


In [40]:
# Remove duplicate rows and reset index
gallery_full = gallery_full.drop_duplicates().reset_index(drop=True)

### Remove row if name missing

In [41]:
gallery_full = gallery_full.dropna(subset=['name'])

### Reorder column names to be clearer

In [42]:
gallery_listings = gallery_full[["id", "name","house_number", "street", "neighborhood_id", "district_id", "postal_code", "website", "opening_hours", "wheelchair", "fee", "latitude", "longitude"]]

In [43]:
gallery_listings.to_csv("../sources/csv_files/gallery_listings.csv", index=False)

### Final Summary of cleaned and Transformed Data

In [44]:
# Shape of dataframe
print(f"Number of rows: {gallery_listings.shape[0]}")
print(f"Number of columns: {gallery_listings.shape[1]}")

Number of rows: 311
Number of columns: 13


In [45]:
# Column list
print("\nRemaining columns:")
print(gallery_listings.columns.tolist())


Remaining columns:
['id', 'name', 'house_number', 'street', 'neighborhood_id', 'district_id', 'postal_code', 'website', 'opening_hours', 'wheelchair', 'fee', 'latitude', 'longitude']


In [46]:
# Missing values check
missing = gallery_listings.isnull().sum()
print("\nMissing values after cleaning and transforming :")
print(missing)


Missing values after cleaning and transforming :
id                   0
name                 0
house_number        95
street              23
neighborhood_id      0
district_id          0
postal_code         24
website            168
opening_hours      168
wheelchair         193
fee                172
latitude             0
longitude            0
dtype: int64


In [47]:
missing_percent = gallery_listings.isnull().mean() * 100
print(missing_percent.sort_values(ascending=False))

wheelchair         62.057878
fee                55.305466
website            54.019293
opening_hours      54.019293
house_number       30.546624
postal_code         7.717042
street              7.395498
id                  0.000000
name                0.000000
neighborhood_id     0.000000
district_id         0.000000
latitude            0.000000
longitude           0.000000
dtype: float64


In [48]:
gallery_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 311 entries, 0 to 321
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               311 non-null    object 
 1   name             311 non-null    object 
 2   house_number     216 non-null    object 
 3   street           288 non-null    object 
 4   neighborhood_id  311 non-null    object 
 5   district_id      311 non-null    object 
 6   postal_code      287 non-null    object 
 7   website          143 non-null    object 
 8   opening_hours    143 non-null    object 
 9   wheelchair       118 non-null    object 
 10  fee              139 non-null    object 
 11  latitude         311 non-null    float64
 12  longitude        311 non-null    float64
dtypes: float64(2), object(11)
memory usage: 34.0+ KB


In [49]:
gallery_listings.head()

,id,name,house_number,street,neighborhood_id,district_id,postal_code,website,opening_hours,wheelchair,fee,latitude,longitude
0,301107444,atelier achim kühn,6,richter straße,0908,11009009,12524,NaN,NaN,NaN,NaN,52.409750,13.571342
1,370766098,galerie zandi,57,mommsen straße,0401,11004004,10629,NaN,mo-fr 11:00-18:00; sa 11:00-16:00,no,False,52.503135,13.313631
2,410135935,studio für gestaltung,31,bürkner straße,0801,11008008,12047,NaN,"th,fr 13:00-18:00; sa 12:00-16:00",yes,False,52.493327,13.427490
3,410692505,la girafe,NaN,NaN,0202,11002002,None,NaN,NaN,no,False,52.492914,13.437057
4,410745800,kronenboden,16,schweden straße,0106,11001001,13357,NaN,NaN,yes,NaN,52.555091,13.375326


## 🖼️🗃️ Step 3: Populate Database

### Import extra Libraries

In [50]:
import psycopg2
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

### Credentials

In [ ]:
user_name=''
password=''

### Create the connection

In [52]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

### Use table tested above with contraints

In [53]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.galleries (
    id VARCHAR(20) PRIMARY KEY,  
    name VARCHAR(200) NOT NULL,
    house_number VARCHAR(20),           
    street VARCHAR(200),
    neighborhood_id VARCHAR(20),
    district_id VARCHAR(200) NOT NULL,
    postal_code VARCHAR(20),
    website VARCHAR(250),
    opening_hours VARCHAR(200),              
    wheelchair VARCHAR(50),
    fee VARCHAR(50),
    latitude DECIMAL(9,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    CONSTRAINT district_id_fk FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

### Execute the create table query

In [54]:
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction
    print("Table 'galleries' created or already exists.")  

Table 'galleries' created or already exists.


### Write the table to the layered Database using to.sql()

In [55]:
gallery_listings.to_sql(
    'galleries',      
    engine,
    schema=schema,
    if_exists='append', # ✅ keeps table, just inserts data
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!


### Query the table with SQL

In [56]:
query = f"""
SELECT * 
FROM berlin_source_data.galleries
LIMIT 10;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,id,name,house_number,street,neighborhood_id,district_id,postal_code,website,opening_hours,wheelchair,fee,latitude,longitude
0,301107444,atelier achim kühn,6,richter straße,0908,11009009,12524,None,None,None,None,52.409750,13.571342
1,370766098,galerie zandi,57,mommsen straße,0401,11004004,10629,None,mo-fr 11:00-18:00; sa 11:00-16:00,no,false,52.503135,13.313631
2,410135935,studio für gestaltung,31,bürkner straße,0801,11008008,12047,None,"th,fr 13:00-18:00; sa 12:00-16:00",yes,false,52.493327,13.427490
3,410692505,la girafe,None,None,0202,11002002,None,None,None,no,false,52.492914,13.437057
4,410745800,kronenboden,16,schweden straße,0106,11001001,13357,None,None,yes,None,52.555091,13.375326
5,511117580,galería lunasol,35,schierker straße,0801,11008008,12051,None,"tu-th 11:00-22:00; fr,sa 11:00-23:00",yes,false,52.470012,13.432264
6,543623547,errant sound,20,runge straße,0101,11001001,10179,None,None,None,None,52.512362,13.419821
7,545738013,saarländische galerie,None,am festungs graben,0101,11001001,10117,https://www.saarlaendische-galerie.eu/,None,None,None,52.518549,13.395823
8,569103989,atelier freier fotografen,None,weisbach straße,0201,11002002,10249,https://www.aff-galerie.de/,"sa,su 15:00-18:00",no,false,52.524655,13.451143
9,734960879,alex xandaton,None,brunnen straße,0101,11001001,10119,None,"""by appointment""",None,false,52.530714,13.400648


### Check all gallery districts exist

In [59]:
query = f"""
SELECT g.district_id, di.district_id, COUNT(*) as gallery_count
FROM berlin_source_data.galleries g
JOIN berlin_source_data.districts di
ON g.district_id = di.district_id
GROUP BY g.district_id, di.district_id
ORDER BY g.district_id;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,district_id,district_id,gallery_count
0,11001001,11001001,109
1,11002002,11002002,53
2,11003003,11003003,33
3,11004004,11004004,47
4,11005005,11005005,3
5,11006006,11006006,7
6,11007007,11007007,19
7,11008008,11008008,25
8,11009009,11009009,5
9,11010010,11010010,3
